In [1]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
import numpy as np

import nltk
nltk.download('punkt')

c:\Users\liching\miniconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\liching\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv("../Dataset/meitei_eng/train.csv")
data.head()

,Unnamed: 0,eng,mani
0,0,in chapter 3 arjun said lord,ꯑꯙ꯭ꯌꯥꯌ ꯳ꯗꯥ ꯑꯔꯖꯨꯅꯅꯥ ꯍꯥꯌꯈꯤ ꯚꯒꯕꯥꯟ꯫
1,1,why do you cast me into dreadful acts when you...,ꯂꯧꯁꯤꯡꯒꯤ ꯂꯝꯕꯤ ꯑꯁꯤ ꯍꯦꯟꯅꯥ ꯐꯩ ꯍꯥꯌꯅꯥ ꯊꯥꯖꯔꯀꯄꯥ ꯃꯇꯃꯗꯥ ...
2,2,he found the way of knowledge easier to practi...,ꯃꯍꯥꯛꯅꯥ ꯂꯧꯁꯤꯡꯒꯤ ꯂꯝꯕꯤ ꯑꯁꯤ ꯆꯠꯅꯍꯅꯕꯗꯥ ꯍꯦꯟꯅꯥ ꯂꯥꯌꯕꯥ ꯎ...
3,3,so there is profit in both success and failure,ꯃꯔꯝ ꯑꯗꯨꯅꯥ ꯃꯥꯌ ꯄꯥꯀꯄꯥ ꯑꯃꯁꯨꯡ ꯃꯥꯌ ꯄꯥꯀꯄꯥ ꯑꯅꯤꯃꯛꯇꯥ ꯀꯥ...
4,4,but by now he understands well that action a p...,ꯑꯗꯨꯕꯨ ꯍꯧꯖꯤꯛ ꯐꯥꯑꯣꯕꯗꯥ ꯃꯍꯥꯛꯅꯥ ꯐꯖꯅꯥ ꯈꯉꯂꯦ ꯃꯗꯨꯗꯤ ꯑꯦꯛ...


In [3]:
model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModel.from_pretrained("google/flan-t5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
mani_tokenizer = tokenizer.train_new_from_iterator(data['mani'], 32100)

In [8]:
# testing

inputs = [data["eng"].iloc[ex] for ex in data.index]
targets = [data["mani"].iloc[ex] for ex in data.index]
max_length = 512
model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
mni_token = mani_tokenizer(targets, max_length=max_length, truncation=True)

# working

In [38]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=False)
val_data.reset_index(inplace= True)

In [40]:
# build a dataset for training model.
def preprocess_function(dataset, en_tokenizer, mn_tokenizer):
    inputs = [dataset["eng"].iloc[ex] for ex in dataset.index]
    targets = [dataset["mani"].iloc[ex] for ex in dataset.index]
    # 
    model_inputs = en_tokenizer(inputs, truncation=True, padding=True, max_length= 512, return_tensors="pt")
    labels = mn_tokenizer(targets, truncation=True, padding=True, max_length= 512, return_tensors="pt")['input_ids']
 
    model_inputs["labels"] = labels
    
    return model_inputs

train_set = preprocess_function(train_data, tokenizer, mani_tokenizer)
val_set = preprocess_function(val_data, tokenizer, mani_tokenizer)


In [52]:
# seq2seq dataset loader
class Seq2SeqDataset(Dataset):
    def __init__(self, encoder_input_ids,encoder_attention_mask, decoder_input_ids):
        self.encoder_input_ids = encoder_input_ids
        self.encoder_attention_ids = encoder_attention_mask
        self.decoder_input_ids = decoder_input_ids

    def __len__(self):
        return len(self.encoder_input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encoder_input_ids[idx],
            "attention_mask": self.encoder_attention_ids[idx],
            "labels": self.decoder_input_ids[idx],
        }

# Assuming train_X and val_X are tuples/lists of encoder and decoder inputs respectively
train_dataset = Seq2SeqDataset(train_set['input_ids'], train_set['attention_mask'], train_set['labels'])
eval_dataset = Seq2SeqDataset(val_set['input_ids'], val_set['attention_mask'] ,val_set['labels'])

# # training args
# training_args = TrainingArguments(
#     output_dir="../meitei_eng/output", # output directory for saving model checkpoints and logs.
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     num_train_epochs=10,
#     evaluation_strategy="epoch", # evaluate after every epoch.
#     save_total_limit=2,
#     eval_steps=500,
#     logging_steps=500,
#     learning_rate=5e-5,
#     save_steps=500, 
# )

# # Instantiate Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset
# )

# trainer.train()


In [16]:
#############

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_metric


In [41]:
args = Seq2SeqTrainingArguments(
    output_dir= "../meitei_eng/output",
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size= 1,
    per_device_eval_batch_size= 1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs= 5,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model= "rouge1",
    report_to= "tensorboard"
)

metric = load_metric("rouge", trust_remote_code=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = mani_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, mani_tokenizer.pad_token_id)
    decoded_labels = mani_tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [54]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init= model_init,
    args=args,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    compute_metrics=compute_metrics
)

In [26]:
# Start TensorBoard before training to monitor it in progress
# model_dir = r"../meitei_eng/output"
# %load_ext tensorboard
# %tensorboard --logdir '{model_dir}'/runs

In [55]:
trainer.train()

                                        
  0%|          | 0/7075 [10:49<?, ?it/s]            c:\Users\liching\miniconda3\envs\torch\Lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'loss': 4.6413, 'grad_norm': 0.870436429977417, 'learning_rate': 3.945724381625442e-05, 'epoch': 0.07}




































































































































































































































































































































































                                        
                                                 

  0%|          | 0/7075 [21:36<?, ?it/s]         



{'eval_loss': 2.936095952987671, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 1.5565, 'eval_runtime': 646.9926, 'eval_samples_per_second': 0.547, 'eval_steps_per_second': 0.547, 'epoch': 0.07}


                                        
  0%|          | 0/7075 [26:24<?, ?it/s]            

...



































































































































































































































































































































































                                        
                                                 

  0%|          | 0/7075 [39:29<?, ?it/s]         



{'eval_loss': 2.6182844638824463, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 0.3757, 'eval_runtime': 784.4679, 'eval_samples_per_second': 0.451, 'eval_steps_per_second': 0.451, 'epoch': 0.14}


                                        
  0%|          | 0/7075 [44:52<?, ?it/s]            c:\Users\liching\miniconda3\envs\torch\Lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'loss': 1.0304, 'grad_norm': 0.44556471705436707, 'learning_rate': 3.832650176678446e-05, 'epoch': 0.21}


KeyboardInterrupt: 

In [4]:
from transformers import AutoModelForSeq2SeqLM

# Provide the path to the directory containing the checkpoint files
checkpoint_path = "../meitei_eng\output\checkpoint-200"

# Load the checkpoint model
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)


In [5]:
text = "why do you cast me into dreadful acts when you believe that the way of knowledge is superior"

tokinzed_enlish = tokenizer(text= text, return_tensors= 'pt')
print(tokinzed_enlish)

{'input_ids': tensor([[ 572,  103,   25, 4061,  140,  139,    3,   26, 5236, 1329, 6775,  116,
           25,  857,   24,    8,  194,   13, 1103,   19, 4784,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [7]:
model.eval()

# generated_ids = model.generate(
#     **tokinzed_enlish,
#     max_length=512,  
#     num_return_sequences=1,  # Number of sequences to generate
#     early_stopping=True  # Stop generation when the model predicts the EOS token
# )
generated_ids = model.generate(
    **tokinzed_enlish)

# Decode the generated sequences
generated_text = mani_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)


c:\Users\liching\miniconda3\envs\torch\Lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
print(generated_text)

In [9]:
generated_ids

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [70]:
gf = mani_tokenizer.decode(train_set['labels'][0], skip_special_tokens=True)
print(gf)

ꯑꯙ꯭ꯌꯥꯌ ꯳ꯗꯥ ꯑꯔꯖꯨꯅꯅꯥ ꯍꯥꯌꯈꯤ ꯚꯒꯕꯥꯟ꯫
